In [ ]:
from huggingface_hub import login

hf_token = "please add your token"  
login(token=hf_token)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint_path = "/kaggle/input/fintune/content/finetuning_llama3/checkpoint-957"

# Load tokenize
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, trust_remote_code=True)

# Load model with automatic offloading
model = AutoModelForCausalLM.from_pretrained(
    checkpoint_path,
    torch_dtype="auto",  # Automatically choose the best dtype
    device_map="auto",  # Automatically distribute layers between CPU & GPU
    offload_folder="/kaggle/working/offload",  # Offload large layers to disk
    trust_remote_code=True
)

print("Model device mapping:", model.hf_device_map)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading adapter weights from /kaggle/input/fintune/content/finetuning_llama3/checkpoint-957 led to unexpected keys not found in the model: score.weight. 


Model device mapping: {'': 'cpu'}


In [ ]:
def generate_response(prompt, max_length=100):
    # Get the correct device for the model's first layer
    device = next(model.parameters()).device  

    # Move input tensors to the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
prompt = "connection with icon icon dear please setup icon per icon engineers please let other details needed thanks lead"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
import torch.onnx

onnx_model_path = "/kaggle/working/llama3_finetuned.onnx"

# Dummy input for ONNX conversion (must match LLaMA input format)
dummy_input = tokenizer("connection with icon icon dear please setup icon per icon engineers please let other details needed thanks lead", return_tensors="pt")

# Convert the model to ONNX format
torch.onnx.export(
    model,
    (dummy_input["input_ids"], dummy_input["attention_mask"]),
    onnx_model_path,
    input_names=["input_ids", "attention_mask"],
    output_names=["output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "output": {0: "batch_size", 1: "sequence_length"},
    },
    opset_version=17  
)

print(f"ONNX model saved at: {onnx_model_path}")
